In [8]:
import cv2
import numpy as np

def find_homography(src_pts, dst_pts):
    """
    Estimate homography matrix using RANSAC algorithm.
    """
    assert len(src_pts) >= 4 and len(dst_pts) >= 4, "At least 4 corresponding points are required."

    # Compute homography matrix
    homography, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

    # Ensure homography matrix is of the correct type and size
    if homography is not None and homography.shape == (3, 3):
        if homography.dtype != np.float32:
            homography = homography.astype(np.float32)
        return homography
    else:
        return None

def image_stitch(img1, img2):
    # Convert images to Gray
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(img1_gray, None)
    kp2, des2 = sift.detectAndCompute(img2_gray, None)

    # Initialize BFMatcher
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test to select good matches
    good = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good.append(m)
    matches = np.asarray(good)

    if len(matches) >= 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        # Find Homography matrix using RANSAC
        homography = find_homography(src_pts, dst_pts)

        if homography is not None:
            # Warp image1 onto image2
            warped_img1 = cv2.warpPerspective(img1, homography, (img2.shape[1] + img1.shape[1], img2.shape[0]))

            # Combine images
            stitched_image = warped_img1.copy()
            stitched_image[0:img2.shape[0], 0:img2.shape[1]] = img2

            return stitched_image
        else:
            raise AssertionError("Failed to estimate homography matrix.")
    else:
        raise AssertionError("Can’t find enough keypoints for stitching.")

# Initialize camera
cap = cv2.VideoCapture(0)  # Change the index if needed

# Capture the first frame to use as a base
ret, prev_frame = cap.read()
if not ret:
    raise RuntimeError("Failed to capture frame from camera.")

# Calculate end time
end_time = cv2.getTickCount() + 10 * cv2.getTickFrequency()

# Stitch images together
while cv2.getTickCount() < end_time:
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to capture frame from camera.")

    try:
        stitched = image_stitch(prev_frame, frame)
        if stitched is not None:
            cv2.imshow("Stitched Panorama", stitched)
            prev_frame = stitched
    except AssertionError as e:
        print(e)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

# Save the stitched image to a file
cv2.imwrite("stitched_image.jpg", prev_frame)

True